In [ ]:
import pandas as pd
from sqlalchemy import create_engine 
import psycopg2

In [ ]:
automation_file = "Resources/automation_data_by_state.csv"
automation_data_df = pd.read_csv(automation_file)

#take the hyphen out of the code
automation_data_df["SOC"] = automation_data_df['SOC'].astype(str)

automation_data_df["SOC"] = automation_data_df['SOC'].str.replace("-", '')

automation_data_df = automation_data_df.rename(columns={"SOC":"detail_id", "Occupation":"occupation",
                                                        "Probability":"probability"})

automation_data_df .set_index("detail_id", inplace=True)

automation_data_df.head()

In [ ]:
#rename states to match SQL tables
automation_data_df = automation_data_df.rename(columns={"District of Columbia":"District_of_Columbia", "Rhode Island":"Rhode_Island",
                                                        "New Hampshire":"New_Hampshire", "New Jersey": "New_Jersey", "New York": "New_York",
                                                        "North Carolina":"North_Carolina", "North Dakota":"North_Dakota", "South Carolina": "South_Carolina",
                                                        "South Dakota":"South_Dakota", "West Virgina": "West_Virgina"})


In [ ]:
occupation_salary_file = "Resources/occupation_salary.csv"
occupation_salary_df = pd.read_csv(occupation_salary_file)
occupation_salary_df.set_index("OCC_CODE")


In [ ]:
#filter occupation dataframe to only major occupation group id and title
occupation_major_df = occupation_salary_df.loc[occupation_salary_df["OCC_GROUP"] == "major", ["OCC_CODE", "OCC_TITLE"]]

#take the hyphen out of the code
occupation_major_df["OCC_CODE"] = occupation_major_df['OCC_CODE'].astype(str)

occupation_major_df["OCC_CODE"] = occupation_major_df['OCC_CODE'].str.replace("-", '')

occupation_major_df = occupation_major_df.rename(columns={"OCC_CODE":"major_id"})

occupation_major_df.set_index("major_id", inplace=True)

occupation_major_df



In [ ]:
#filter occupation dataframe to only minor occupation group id and title
occupation_minor_df = occupation_salary_df.loc[occupation_salary_df["OCC_GROUP"] == "minor", ["OCC_CODE", "OCC_TITLE"]]

#take the hyphen out of the code
occupation_minor_df["OCC_CODE"] = occupation_minor_df['OCC_CODE'].astype(str)

occupation_minor_df["OCC_CODE"] = occupation_minor_df['OCC_CODE'].str.replace("-", '')

occupation_minor_df = occupation_minor_df.rename(columns={"OCC_CODE":"minor_id"})

#add the associated broad OCC_CODE to each detail row to create link to other tables
#store last digit of the code in a variable
occupation_minor_df["major_id"] = occupation_minor_df['minor_id'].str[:2] + '0000'

#change id columns to integers
occupation_minor_df["major_id"] = occupation_minor_df['major_id'].astype(int)
occupation_minor_df["minor_id"] = occupation_minor_df["minor_id"].astype(int)

#set index to id column
occupation_minor_df.set_index("minor_id", inplace=True)

occupation_minor_df

In [ ]:
#filter occupation dataframe to only broad occupation group id and title
occupation_broad_df = occupation_salary_df.loc[occupation_salary_df["OCC_GROUP"] == "broad",
                                              ["OCC_CODE", "OCC_TITLE"]]

#take the hyphen out of the code
occupation_broad_df["OCC_CODE"] = occupation_broad_df['OCC_CODE'].astype(str)

occupation_broad_df["OCC_CODE"] = occupation_broad_df['OCC_CODE'].str.replace("-", '')

occupation_broad_df = occupation_broad_df.rename(columns={"OCC_CODE":"broad_id"})

#add the associated broad OCC_CODE to each detail row to create link to other tables
#store last digit of the code in a variable
occupation_broad_df["minor_id"] = occupation_broad_df['broad_id'].str[:4] + '00'

#change id columns to integers
occupation_broad_df["broad_id"] = occupation_broad_df['broad_id'].astype(int)
occupation_broad_df["minor_id"] = occupation_broad_df["minor_id"].astype(int)

#set index to id column
occupation_broad_df.set_index("broad_id", inplace=True)

occupation_broad_df

In [ ]:
#filter occupation dataframe to detail occupation data
occupation_detail_df = occupation_salary_df.loc[occupation_salary_df["OCC_GROUP"] == "detailed",
                                               ["OCC_CODE", "OCC_TITLE", "TOT_EMP", "A_MEAN", "A_MEDIAN", "H_MEAN", "H_MEDIAN"]]
occupation_detail_df.head(10)

occupation_detail_df.dtypes

In [ ]:
#take the hyphen out of the code
occupation_detail_df["OCC_CODE"] = occupation_detail_df['OCC_CODE'].astype(str)

occupation_detail_df["OCC_CODE"] = occupation_detail_df['OCC_CODE'].str.replace("-", '')


occupation_detail_df = occupation_detail_df.rename(columns={"OCC_CODE":"detail_id", "TOT_EMP":"total_emp",
                                                            "A_MEAN":"a_mean", "A_MEDIAN":"a_median", "H_MEAN": "h_mean",
                                                            "H_MEDIAN":"h_median"
                                                           })

#add the associated broad OCC_CODE to each detail row to create link to other tables
#store last digit of the code in a variable
occupation_detail_df["broad_id"] = occupation_detail_df['detail_id'].str[:5] + '0'

#change id columns to integers
occupation_detail_df["detail_id"] = occupation_detail_df['detail_id'].astype(int)
occupation_detail_df["broad_id"] = occupation_detail_df["broad_id"].astype(int)

#remove * from rows
occupation_detail_df["h_mean"] = occupation_detail_df["h_mean"].replace("*", "")
occupation_detail_df["h_median"] = occupation_detail_df["h_median"].replace("*", "")
occupation_detail_df["a_median"] = occupation_detail_df["a_median"].replace("*", "")
occupation_detail_df["a_mean"] = occupation_detail_df["a_mean"].replace("*", "")
                                                                            
#remove commas from rows
#convert to string and remove comma, conmvert to int
occupation_detail_df["total_emp"] = occupation_detail_df['total_emp'].astype(str)
occupation_detail_df["total_emp"] = occupation_detail_df['total_emp'].str.replace(",", "")
occupation_detail_df["total_emp"] = occupation_detail_df["total_emp"].astype(int)

#convert to string and remove comma
occupation_detail_df["a_mean"] = occupation_detail_df['a_mean'].astype(str)
occupation_detail_df["a_mean"] = occupation_detail_df['a_mean'].str.replace(",", "")

#convert to string and remove comma
occupation_detail_df["a_median"] = occupation_detail_df["a_median"].astype(str)
occupation_detail_df["a_median"] = occupation_detail_df["a_median"].str.replace(",", "")

#set index to id column
occupation_detail_df.set_index("detail_id", inplace=True)

occupation_detail_df.head(20)



In [ ]:
#confirm datatypes of columns
occupation_detail_df.dtypes

In [ ]:
connection_string = "postgres:Weiser@localhost:5432/AutomationETL"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Confirm tables
engine.table_names()

In [ ]:
#load tables - detail first
occupation_detail_df.to_sql(name='occupation_detail', con=engine, if_exists='append', index=True)

In [ ]:
#load tables
automation_data_df.to_sql(name='automation', con=engine, if_exists='append', index=True)

In [ ]:
#load tables
occupation_major_df.to_sql(name='occupation_major', con=engine, if_exists='append', index=True)

In [ ]:
#load tables
occupation_minor_df.to_sql(name='occupation_minor', con=engine, if_exists='append', index=True)

In [ ]:
#load tables
occupation_broad_df.to_sql(name='occupation_broad', con=engine, if_exists='append', index=True)

In [ ]:
#load tables
occupation_detail_df.to_sql(name='occupation_detail', con=engine, if_exists='append', index=True)